This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
####### DATA PREPARATION #####
#load data
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173         

# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

True
True


In [3]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

num_features = 54

model = Sequential()
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dropout(0.2))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='Adam',
              metrics=['accuracy'])

#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

# Predict!!
y_predict = model.predict(x_predict)

Using TensorFlow backend.


Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 1s 97us/sample - loss: 1.0925 - acc: 0.5590 - val_loss: 0.8748 - val_acc: 0.6270
Epoch 2/250
12096/12096 [==============================] - 1s 75us/sample - loss: 0.8472 - acc: 0.6408 - val_loss: 0.8187 - val_acc: 0.6524
Epoch 3/250
12096/12096 [==============================] - 1s 76us/sample - loss: 0.7966 - acc: 0.6602 - val_loss: 0.7786 - val_acc: 0.6670
Epoch 4/250
12096/12096 [==============================] - 1s 78us/sample - loss: 0.7578 - acc: 0.6784 - val_loss: 0.7420 - val_acc: 0.6928
Epoch 5/250
12096/12096 [==============================] - 1s 77us/sample - loss: 0.7248 - acc: 0.6910 - val_loss: 0.7520 - val_acc: 0.6855
Epoch 6/250
12096/12096 [==============================] - 1s 75us/sample - loss: 0.7070 - acc: 0.7017 - val_loss: 0.6934 - val_acc: 0.7103
Epoch 7/250
12096/12096 [==============================] - 1s 75us/sample - loss: 0.6903 - acc: 0.7046 - val_lo

In [4]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

[1.07184090e-01 8.74214947e-01 1.67411363e-09 1.73665825e-02
 4.83540673e-04 7.50773877e-04 1.12877286e-11] 2
[5.0154299e-01 4.9835441e-01 3.0679756e-13 8.3283815e-08 3.8032544e-05
 6.4508124e-05 3.8560753e-13] 1
[6.2735868e-01 3.7252390e-01 2.7095516e-12 5.5577181e-07 2.6514022e-05
 9.0309237e-05 1.6038811e-12] 1
[6.5296805e-01 3.4663934e-01 2.9422784e-11 1.2675684e-05 2.2789005e-05
 3.5711157e-04 5.5817325e-13] 1
[6.4401954e-01 3.5448074e-01 2.6731692e-10 2.0737956e-04 2.3733333e-05
 1.2685615e-03 1.7369851e-13] 1
[2.9976532e-01 6.9884324e-01 1.5207570e-10 1.7887800e-05 1.2702828e-04
 1.2464229e-03 4.6849040e-14] 2
[3.3833778e-01 6.6090798e-01 4.3819066e-11 3.7185234e-06 1.0798696e-04
 6.4250850e-04 1.2037769e-13] 2
[3.2233235e-01 6.7709613e-01 1.3145157e-11 9.6272663e-07 1.4358820e-04
 4.2698457e-04 1.3843846e-13] 2
[4.1172525e-01 5.8713984e-01 9.9820265e-12 2.0514074e-06 1.9162110e-04
 9.4116689e-04 6.5396392e-14] 2
[3.1496155e-01 6.8416589e-01 1.5745268e-11 1.9297388e-06 1.6209710

In [5]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv